In [1]:
import pandas as pd
#import pandas.io.sql as sqlio
#import sqlalchemy as sql
import psycopg2
#import matplotlib.pyplot as plt
import numpy as np
import os

from dotenv import load_dotenv
#from matplotlib import colors
#from matplotlib.ticker import PercentFormatter
   
from sqlalchemy import create_engine

# load up the variables from the .env file.
load_dotenv()

True

### States Abbreviation Table

In [19]:
# File to Load (Remember to Change These)
file_to_load = "Resources/StatetoAbbrev.csv"
states_data = pd.read_csv(file_to_load)
states_data.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [20]:
states_data['State_id']=states_data.index+1
states_data.head()

,State,Abbrev,Code,State_id
0,Alabama,Ala.,AL,1
1,Alaska,Alaska,AK,2
2,Arizona,Ariz.,AZ,3
3,Arkansas,Ark.,AR,4
4,California,Calif.,CA,5


Resources / Natural Gas

In [21]:
file_to_load = "Resources/Natural Gas Marketed Production-StateRankings.csv"

natural_gas_production = pd.read_csv(file_to_load)

naturalgas_df = pd.merge(states_data,natural_gas_production,how='outer', left_on='Code', right_on='State')

naturalgas_df.drop(columns = ["State_x","State_y","Code","Abbrev","Note: Rankings are based on the full source data values. Excludes Gulf of Mexico federal offshore production."], inplace = True)

naturalgas_df.head()

,State_id,Rank,"Natural Gas Marketed Production, million cu ft"
0,1.0,16.0,139485
1,2.0,12.0,341315
2,3.0,31.0,46
3,4.0,11.0,589973
4,5.0,14.0,202616


In [22]:
file_to_load = "Resources/P2.csv"
production_df = pd.read_csv(file_to_load)

production_df.rename(columns ={"Table P2.  Primary Energy Production Estimates in Trillion Btu, 2018 ":"State",
                               "Unnamed: 1":"Fossil Fuels", "Unnamed: 3":"Natural Gas", "Unnamed: 5":"Crude Oil",
                              "Unnamed: 9":"Biofuels", "Unnamed: 7":"Nuclear Electric Power","Unnamed: 11":"Wood and Waste",
                              "Unnamed: 13":"Other", "Unnamed: 15":"Total"}, inplace = True)

production_df.drop(columns = ['Unnamed: 2','Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8','Unnamed: 10', 'Unnamed: 12', 'Unnamed: 14',
                             'Unnamed: 16'], inplace = True)

production_df.drop([0,1,2,3], inplace = True)


productionfinal_df = pd.merge(states_data,production_df,how='outer', left_on='State', right_on='State')
productionfinal_df.drop(columns=["Abbrev", "State", "Code"], inplace = True)
productionfinal_df.head()

,State_id,Fossil Fuels,Natural Gas,Crude Oil,Nuclear Electric Power,Biofuels,Wood and Waste,Other,Total
0,1.0,370.5,149.6,33.6,412.6,1.7,170.1,105.0,"1,243.0"
1,2.0,13.8,375.3,997.4,0.0,(s),7.3,16.8,"1,410.6"
2,3.0,140.8,(s),0.1,325.1,6.8,8.7,139.5,621.0
3,4.0,0.0,600.4,28.6,133.0,9.5,85.2,30.3,887.1
4,5.0,0.0,228.9,965.3,190.4,35.5,130.5,857.6,"2,408.2"


Renewable Potential energy

In [23]:
# File to Load (Remember to Change These)
file_to_load = "Resources/usretechnicalpotential.csv"

# Read Renewable energy potential file and store into Pandas data frame
renewable_energy_data = pd.read_csv(file_to_load)
renewable_energy_data.columns

Index(['Unnamed: 0', 'urbanUtilityScalePV_GWh', 'urbanUtilityScalePV_GW',
       'urbanUtilityScalePV_km2', 'ruralUtilityScalePV_GWh',
       'ruralUtilityScalePV_GW', 'ruralUtilityScalePV_km2', 'rooftopPV_GWh',
       'rooftopPV_GW', 'CSP_GWh', 'CSP_GW', 'CSP_km2', 'onshoreWind_GWh',
       'onshoreWind_GW', 'onshoreWind_km2', 'offshoreWind_GWh',
       'offshoreWind_GW', 'offshoreWind_km2', 'biopowerSolid_GWh',
       'biopowerSolid_GW', 'biopowerSolid_BDT', 'biopowerGaseous_GWh',
       'biopowerGaseous_GW', 'biopowerGaseous_Tonnes-CH4',
       'geothermalHydrothermal_GWh', 'geothermalHydrothermal_GW',
       'EGSGeothermal_GWh', 'EGSGeothermal_GW', 'hydropower_GWh',
       'hydropower_GW', 'hydropower_countOfSites'],
      dtype='object')

In [24]:

renewable_energy_data.rename(columns={'Unnamed: 0' : 'State'}, inplace=True)
bio_power_potential = renewable_energy_data[['State', 'biopowerSolid_GWh', 'biopowerGaseous_GWh']]
bio_power_potential.head()

,State,biopowerSolid_GWh,biopowerGaseous_GWh
0,Alabama,11193,1533
1,Alaska,513,61
2,Arizona,1087,837
3,Arkansas,14381,1063
4,California,12408,15510


In [25]:

combined_df = pd.merge(states_data,bio_power_potential,how='outer', on='State')
combined_df.head()

,State,Abbrev,Code,State_id,biopowerSolid_GWh,biopowerGaseous_GWh
0,Alabama,Ala.,AL,1,11193,1533
1,Alaska,Alaska,AK,2,513,61
2,Arizona,Ariz.,AZ,3,1087,837
3,Arkansas,Ark.,AR,4,14381,1063
4,California,Calif.,CA,5,12408,15510


In [26]:
m = len(combined_df.index)
print(m)

51


In [27]:
bio_power = combined_df[['State_id', 'biopowerSolid_GWh', 'biopowerGaseous_GWh']]
bio_power.set_index('State_id', inplace=True)
bio_power.head()

,biopowerSolid_GWh,biopowerGaseous_GWh
State_id,,
1,11193,1533
2,513,61
3,1087,837
4,14381,1063
5,12408,15510


### Total Net electricity

In [28]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Total Net Electricity Generation-StateRankings.csv"

# Read Renewable energy potential file and store into Pandas data frame
total_electricity_data = pd.read_csv(file_to_load)
total_electricity_data.head()

,Rank,State,"Total Net Electricity Generation, thousand MWh",Note: Rankings are based on the full source data values.
0,1,TX,38524,NaN
1,2,FL,20787,NaN
2,3,PA,16521,NaN
3,4,CA,15465,NaN
4,5,IL,12997,NaN


In [29]:
total_electricity = total_electricity_data.drop(columns='Note: Rankings are based on the full source data values.')
total_electricity.rename(columns={'Total Net Electricity Generation, thousand MWh': 'Total_Net(GWh)'}, inplace=True)
total_electricity.head()

,Rank,State,Total_Net(GWh)
0,1,TX,38524
1,2,FL,20787
2,3,PA,16521
3,4,CA,15465
4,5,IL,12997


In [30]:
combined_df = pd.merge(states_data,total_electricity,how='outer', left_on='Code', right_on='State')
combined_df.head()

,State_x,Abbrev,Code,State_id,Rank,State_y,Total_Net(GWh)
0,Alabama,Ala.,AL,1,7,AL,10575
1,Alaska,Alaska,AK,2,48,AK,476
2,Arizona,Ariz.,AZ,3,10,AZ,9350
3,Arkansas,Ark.,AR,4,31,AR,3532
4,California,Calif.,CA,5,4,CA,15465


In [31]:
n = len(combined_df.index)
for i in range(n):
    if combined_df.loc[i,'Code'] != combined_df.loc[i, 'State_y']:
        print(i, combined_df.loc[i,'Code'], combined_df.loc[i, 'State_y'])

In [32]:

total_electricity_df = combined_df[['State_id', 'Rank', 'Total_Net(GWh)']]
total_electricity_df.set_index('State_id', inplace=True)
total_electricity_df.head()

,Rank,Total_Net(GWh)
State_id,,
1,7,10575
2,48,476
3,10,9350
4,31,3532
5,4,15465


Coal and Crude Oil

In [36]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Coal Production-StateRankings.csv"
Coal_Production = pd.read_csv(file_to_load)
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",Note: Rankings are based on the full source data values.
0,1,WY,304188,NaN
1,2,WV,95365,NaN
2,3,PA,49883,NaN
3,4,IL,49563,NaN
4,5,KY,39567,NaN


In [40]:
Coal_Production.drop(columns=["Note: Rankings are based on the full source data values."],inplace=True)
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons"
0,1,WY,304188
1,2,WV,95365
2,3,PA,49883
3,4,IL,49563
4,5,KY,39567


In [43]:

Coal_combined_df = pd.merge(states_data,Coal_Production,how='inner', left_on='Code', right_on='State')
Coal_combined_df.head()

,State_x,Abbrev,Code,State_id,Rank,State_y,"Coal Production, thousand short tons"
0,Alabama,Ala.,AL,1,10,AL,14783
1,Alaska,Alaska,AK,2,20,AK,902
2,Arizona,Ariz.,AZ,3,16,AZ,6550
3,Colorado,Colo.,CO,6,11,CO,14026
4,Illinois,Ill.,IL,14,4,IL,49563


In [44]:
Coal_combined_df.drop(columns=["State_x", "Abbrev", "Code"],inplace=True)
Coal_combined_df.head()

,State_id,Rank,State_y,"Coal Production, thousand short tons"
0,1,10,AL,14783
1,2,20,AK,902
2,3,16,AZ,6550
3,6,11,CO,14026
4,14,4,IL,49563


In [37]:

# File to Load (Remember to Change These)
file_to_load = "Resources/Crude Oil Production-StateRankings.csv"
CrudeOil_Production = pd.read_csv(file_to_load)
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day",Note: Rankings are based on the full source data values. Excludes federal offshore production.
0,1,TX,4395,NaN
1,2,NM,885,NaN
2,3,ND,862,NaN
3,4,CO,471,NaN
4,5,AK,404,NaN


In [38]:

CrudeOil_Production.drop(columns=["Note: Rankings are based on the full source data values. Excludes federal offshore production."],inplace=True)
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day"
0,1,TX,4395
1,2,NM,885
2,3,ND,862
3,4,CO,471
4,5,AK,404


In [45]:
Coal_Crude_combined_df = pd.merge(states_data,CrudeOil_Production,how='inner', left_on='Code', right_on='State')
Coal_Crude_combined_df.head()

,State_x,Abbrev,Code,State_id,Rank,State_y,"Crude Oil Production, thousand barrels per day"
0,Alabama,Ala.,AL,1,20,AL,6
1,Alaska,Alaska,AK,2,5,AK,404
2,Arizona,Ariz.,AZ,3,31,AZ,0
3,Arkansas,Ark.,AR,4,18,AR,10
4,California,Calif.,CA,5,6,CA,392


In [46]:
Coal_Crude_combined_df.drop(columns=["State_x", "Abbrev", "Code"],inplace=True)
Coal_Crude_combined_df.head()

,State_id,Rank,State_y,"Crude Oil Production, thousand barrels per day"
0,1,20,AL,6
1,2,5,AK,404
2,3,31,AZ,0
3,4,18,AR,10
4,5,6,CA,392


Total Energy 

In [48]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Total Energy Consumed per Capita-StateRankings.csv"

# Read Renewable energy potential file and store into Pandas data frame
Totalenergyconsumed = pd.read_csv(file_to_load)
Totalenergyconsumed.head()

,Rank,State,"Total Energy Consumed per Capita, million Btu",Note: Rankings are based on the full source data values.
0,1,WY,967,NaN
1,2,LA,945,NaN
2,3,ND,872,NaN
3,4,AK,830,NaN
4,5,IA,513,NaN


In [50]:
Totalenergyconsumed = Totalenergyconsumed[['Rank', 'State', 'Total Energy Consumed per Capita, million Btu']]

Totalenergycon_clean = pd.merge(Totalenergyconsumed, states_data,how='inner',left_on='State', right_on='Code')

Totalenergycon_clean = Totalenergycon_clean[['State_id','Rank','Total Energy Consumed per Capita, million Btu']]

Totalenergycon_clean.set_index("State_id", inplace=True)

Totalenergycon_clean.head()

,Rank,"Total Energy Consumed per Capita, million Btu"
State_id,,
51,1,967
19,2,945
35,3,872
2,4,830
16,5,513


In [51]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Total Energy Production-StateRankings.csv"

# Read Renewable energy potential file and store into Pandas data frame
Totalenergyproduction = pd.read_csv(file_to_load)
Totalenergyproduction.head()

,Rank,State,"Total Energy Production, trillion Btu",Note: Rankings are based on the full source data values. Excludes federal offshore production.
0,1,TX,20421,NaN
1,2,PA,8987,NaN
2,3,WY,7718,NaN
3,4,OK,4881,NaN
4,5,WV,4770,NaN


In [52]:
Totalenergyproduction = Totalenergyproduction[['Rank', 'State', 'Total Energy Production, trillion Btu']]

Totalenergyprod_clean = pd.merge(Totalenergyproduction, states_data,how='inner',left_on='State', right_on='Code')

Totalenergyprod_clean = Totalenergyprod_clean[['State_id','Rank','Total Energy Production, trillion Btu']]

Totalenergyprod_clean.set_index("State_id", inplace=True)

Totalenergyprod_clean.head()

,Rank,"Total Energy Production, trillion Btu"
State_id,,
44,1,20421
39,2,8987
51,3,7718
37,4,4881
49,5,4770


Loading the data into SQL DB

In [53]:
# set the variables for the pull from SQL.  username and password are in an .env file.  You will need those to make this run
# on your machine.

host = 'localhost'
port = 5432
dbname='ETL_Project'
username = os.environ.get('USER_NAME')
pwd = os.environ.get('PASSWORD')

In [55]:

# set up the connection to the database "ETL_Project"
connection_string = f'{username}:{pwd}@{host}:{port}/{dbname}'
engine = create_engine(f'postgresql://{connection_string}')

In [56]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  database "ETL_Project" does not exist

(Background on this error at: http://sqlalche.me/e/13/e3q8)

EJ

In [ ]:
#states_data.to_sql(name='State_Abb', con=engine, if_exists='append', index=False)
#total_electricity_df.to_sql(name='Total_Net_Electricity', con=engine, if_exists='append', index=False)
renewable_energy_data.to_sql(name='Bio_Power_Potential', con=engine, if_exists='append', index=False)

Niral

Teshanee

Bill